# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Read it in here
vacay_df = pd.read_csv('output_data/cities.csv')
vacay_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,5.710000e+02
mean,285.000000,19.256726,17.626464,59.913678,71.730298,54.810858,7.625464,1.650419e+09
std,164.977776,33.161820,89.578172,20.178032,21.560623,40.756779,5.443808,8.265086e+01
min,0.000000,-54.800000,-175.200000,-9.170000,1.000000,0.000000,0.000000,1.650419e+09
25%,142.500000,-8.211650,-66.047300,44.935000,62.000000,11.000000,3.530000,1.650419e+09
50%,285.000000,21.461200,20.066700,64.690000,76.000000,64.000000,6.510000,1.650419e+09
75%,427.500000,47.870850,99.967850,76.200000,88.000000,100.000000,10.360000,1.650419e+09
max,570.000000,78.218600,178.416700,91.380000,100.000000,100.000000,32.210000,1.650419e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations variable.
# Also, store the 'Humidity' values into another variable that will later be passed
# into the gmaps heatmapper
locations = vacay_df[["Lat", "Lng"]].astype(float)
humidity = vacay_df["Humidity"].astype(float)

# Create a map figure
fig = gmaps.figure()

# Create a heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# create new dataframe by filtering our 'vacay_df' on several columns:
# Max Temp, Wind Speed, and Cloudiness 
my_df = vacay_df[(vacay_df['Max Temp'] >= 75) & (vacay_df['Max Temp'] <= 90)
                 & (vacay_df['Wind Speed'] < 10)
                 & (vacay_df['Cloudiness'] < 15)]
my_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Nuevo Progreso,23.1667,-97.9500,76.53,71,6,1.01,MX,1650419112
15,15,Cape Town,-33.9258,18.4232,75.11,82,0,5.75,ZA,1650418963
36,36,Vao,-22.6667,167.4833,75.78,69,0,6.46,NC,1650419122
82,82,Pedernales,18.0384,-71.7440,79.20,73,12,3.49,DO,1650419136
126,126,Flinders,-34.5833,150.8552,75.38,58,0,7.00,AU,1650419150
139,139,Saint-Pierre,-21.3393,55.4781,76.68,81,0,4.61,RE,1650419154
142,142,Bilma,18.6853,12.9164,75.51,14,7,6.93,NE,1650419155
161,161,Bareilly,28.3500,79.4167,87.39,19,3,3.94,IN,1650419161
173,173,Wollongong,-34.4333,150.8833,75.25,57,0,4.05,AU,1650419100
202,202,Sur,22.5667,59.5289,79.65,65,0,8.75,OM,1650419174


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = my_df.copy()
hotel_df["Hotel"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
2,2,Nuevo Progreso,23.1667,-97.9500,76.53,71,6,1.01,MX,1650419112,
15,15,Cape Town,-33.9258,18.4232,75.11,82,0,5.75,ZA,1650418963,
36,36,Vao,-22.6667,167.4833,75.78,69,0,6.46,NC,1650419122,
82,82,Pedernales,18.0384,-71.7440,79.20,73,12,3.49,DO,1650419136,
126,126,Flinders,-34.5833,150.8552,75.38,58,0,7.00,AU,1650419150,
139,139,Saint-Pierre,-21.3393,55.4781,76.68,81,0,4.61,RE,1650419154,
142,142,Bilma,18.6853,12.9164,75.51,14,7,6.93,NE,1650419155,
161,161,Bareilly,28.3500,79.4167,87.39,19,3,3.94,IN,1650419161,
173,173,Wollongong,-34.4333,150.8833,75.25,57,0,4.05,AU,1650419100,
202,202,Sur,22.5667,59.5289,79.65,65,0,8.75,OM,1650419174,


In [11]:
params = {
    "types": "hotel",
    "radius": 5000,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel"] = response["results"][1]["name"]
        print("success!")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt><b>Name</b></dt><dd>{Hotel}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))